In [1]:
#initialize model
from loss import ContrastiveLoss
import torch
from ml_training import *
from dl_training import *


In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Resnet 50

In [3]:
from models.resnet import ResNet50
criterion=ContrastiveLoss()
res50_model=ResNet50()
opt=torch.optim.Adam(params=res50_model.parameters(),lr=0.0001)
print(count_parameters(res50_model))

24577408


In [4]:
#train model
res50_model_trained=train_dl(50,res50_model,"cuda",criterion,opt)

In [5]:
#extract features
fracture=get_features('../dataset/train/Fracture',res50_model_trained)
normal= get_features('../dataset/train/Normal',res50_model_trained)

In [6]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [7]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [8]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.008287,0.118387,0.018148,0.014547
score_time,0.007573,0.010372,0.007786,0.016194
test_accuracy,0.815330,0.821569,0.791622,0.815686
train_accuracy,0.896184,1.000000,0.881123,0.805878
test_precision_macro,0.844632,0.844360,0.808635,0.830094
train_precision_macro,0.900889,1.000000,0.879435,0.820912
test_recall_macro,0.833797,0.831541,0.794211,0.834060
train_recall_macro,0.908843,1.000000,0.887107,0.822573
test_f1_macro,0.804429,0.810685,0.777894,0.803919
train_f1_macro,0.895841,1.000000,0.880010,0.805613


In [9]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',res50_model_trained)
test_normal= get_features('../dataset/test/Normal',res50_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [10]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [11]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.58      0.58      0.58        12
           1       0.67      0.67      0.67        15

    accuracy                           0.63        27
   macro avg       0.62      0.62      0.62        27
weighted avg       0.63      0.63      0.63        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.54      0.58      0.56        12
           1       0.64      0.60      0.62        15

    accuracy                           0.59        27
   macro avg       0.59      0.59      0.59        27
weighted avg       0.60      0.59      0.59        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        12
           1       0.80      0.80      0.80        15

    accuracy                           0.78        

# mobile net

In [12]:
from models.mobilenet import MobileNetV2
criterion=ContrastiveLoss()
mobilenet=MobileNetV2()
opt=torch.optim.Adam(params=mobilenet.parameters(),lr=0.0001)
print(count_parameters(mobilenet))

2879168


In [13]:
#train model
mobilenet_trained=train_dl(50,mobilenet,"cuda",criterion,opt)


In [14]:
#extract features
fracture=get_features('../dataset/train/Fracture',mobilenet_trained)
normal= get_features('../dataset/train/Normal',mobilenet_trained)

In [15]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [16]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [17]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.009816,0.118514,0.021947,0.014170
score_time,0.006596,0.010030,0.006425,0.013784
test_accuracy,0.602496,0.619964,0.607843,0.590018
train_accuracy,0.911005,1.000000,1.000000,0.730383
test_precision_macro,0.592568,0.607448,0.594015,0.579702
train_precision_macro,0.910100,1.000000,1.000000,0.730829
test_recall_macro,0.592444,0.603947,0.589662,0.565827
train_recall_macro,0.911953,1.000000,1.000000,0.711287
test_f1_macro,0.592278,0.601139,0.583623,0.562908
train_f1_macro,0.909256,1.000000,1.000000,0.713895


In [18]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',mobilenet_trained)
test_normal= get_features('../dataset/test/Normal',mobilenet_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [19]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [20]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.69      0.75      0.72        12
           1       0.79      0.73      0.76        15

    accuracy                           0.74        27
   macro avg       0.74      0.74      0.74        27
weighted avg       0.74      0.74      0.74        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.67      0.50      0.57        12
           1       0.67      0.80      0.73        15

    accuracy                           0.67        27
   macro avg       0.67      0.65      0.65        27
weighted avg       0.67      0.67      0.66        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.67      0.50      0.57        12
           1       0.67      0.80      0.73        15

    accuracy                           0.67        

# Squeeze net

In [21]:
from models.squeezenet import SqueezeNetV1
criterion=ContrastiveLoss()
squeezenet=SqueezeNetV1()
opt=torch.optim.Adam(params=squeezenet.parameters(),lr=0.0001)
print(count_parameters(squeezenet))

984000


In [22]:
#train model
squeezenet_trained=train_dl(50,squeezenet,"cuda",criterion,opt)

In [23]:
#extract features
fracture=get_features('../dataset/train/Fracture',squeezenet_trained)
normal= get_features('../dataset/train/Normal',squeezenet_trained)

In [24]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [25]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [26]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.010568,0.102596,0.019949,0.024237
score_time,0.006738,0.010372,0.007579,0.015765
test_accuracy,0.754189,0.707130,0.706774,0.731194
train_accuracy,0.819424,0.799784,0.826885,0.738095
test_precision_macro,0.783860,0.720349,0.744494,0.744911
train_precision_macro,0.845459,0.795902,0.827749,0.733569
test_recall_macro,0.768308,0.704962,0.689398,0.727632
train_recall_macro,0.837286,0.798402,0.815499,0.734911
test_f1_macro,0.751386,0.697600,0.689153,0.722365
train_f1_macro,0.818551,0.796206,0.819483,0.733378


In [27]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',squeezenet_trained)
test_normal= get_features('../dataset/test/Normal',squeezenet_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [28]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [29]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.50      0.67      0.57        12
           1       0.64      0.47      0.54        15

    accuracy                           0.56        27
   macro avg       0.57      0.57      0.55        27
weighted avg       0.58      0.56      0.55        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.62      0.67      0.64        12
           1       0.71      0.67      0.69        15

    accuracy                           0.67        27
   macro avg       0.66      0.67      0.66        27
weighted avg       0.67      0.67      0.67        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.55      0.50      0.52        12
           1       0.62      0.67      0.65        15

    accuracy                           0.59        

# My Model V1

In [30]:
from models.mymodels import SqueezeNetV2
criterion=ContrastiveLoss()
v2_model=SqueezeNetV2()
opt=torch.optim.Adam(params=v2_model.parameters(),lr=0.0001)
print(count_parameters(v2_model))

5781184


In [31]:
#train model
v2_model_trained=train_dl(50,v2_model,"cuda",criterion,opt)

In [32]:
#extract features
fracture=get_features('../dataset/train/Fracture',v2_model_trained)
normal= get_features('../dataset/train/Normal',v2_model_trained)

In [33]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [34]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [35]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.008977,0.088437,0.020541,0.016157
score_time,0.008377,0.010374,0.007578,0.015666
test_accuracy,0.803209,0.791087,0.748841,0.796970
train_accuracy,0.814878,0.781761,0.762224,0.778754
test_precision_macro,0.808931,0.801821,0.774743,0.809242
train_precision_macro,0.813288,0.785481,0.756877,0.780961
test_recall_macro,0.805301,0.796917,0.741278,0.801917
train_recall_macro,0.817755,0.788192,0.754281,0.785091
test_f1_macro,0.800354,0.787391,0.735742,0.794126
train_f1_macro,0.812672,0.780126,0.755318,0.777231


In [36]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',v2_model_trained)
test_normal= get_features('../dataset/test/Normal',v2_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [37]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [38]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.85      0.73      0.79        15

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.79      0.78      0.78        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.85      0.73      0.79        15

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.79      0.78      0.78        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        12
           1       0.80      0.80      0.80        15

    accuracy                           0.78        

# My Model V2

In [39]:
from models.mymodels import SqueezeNetV3
criterion=ContrastiveLoss()
v3_model=SqueezeNetV3()
opt=torch.optim.Adam(params=v3_model.parameters(),lr=0.0001)
print(count_parameters(v3_model))

774768


In [40]:
#train model
v3_model_trained=train_dl(50,v3_model,"cuda",criterion,opt)

In [41]:
#extract features
fracture=get_features('../dataset/train/Fracture',v3_model_trained)
normal= get_features('../dataset/train/Normal',v3_model_trained)

In [42]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [43]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [44]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.008578,0.071814,0.024744,0.014568
score_time,0.006582,0.007580,0.006788,0.015359
test_accuracy,0.814795,0.839037,0.796613,0.839216
train_accuracy,0.837503,0.837434,0.840453,0.869059
test_precision_macro,0.859426,0.879729,0.850465,0.880347
train_precision_macro,0.853123,0.838707,0.837316,0.872352
test_recall_macro,0.831391,0.846805,0.794925,0.850827
train_recall_macro,0.853341,0.845010,0.842268,0.879605
test_f1_macro,0.806361,0.829778,0.781694,0.828320
train_f1_macro,0.837154,0.836258,0.838146,0.868413


In [45]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',v3_model_trained)
test_normal= get_features('../dataset/test/Normal',v3_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [46]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [47]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.62      0.83      0.71        12
           1       0.82      0.60      0.69        15

    accuracy                           0.70        27
   macro avg       0.72      0.72      0.70        27
weighted avg       0.73      0.70      0.70        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.85      0.73      0.79        15

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.79      0.78      0.78        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        12
           1       0.75      0.80      0.77        15

    accuracy                           0.74        

# My Model V3

In [48]:
from models.mymodels import SqueezeNetV4
criterion=ContrastiveLoss()
v4_model=SqueezeNetV4()
opt=torch.optim.Adam(params=v4_model.parameters(),lr=0.0001)
print(count_parameters(v4_model))

774768


In [49]:
#train model
v4_model_trained=train_dl(50,v4_model,"cuda",criterion,opt)

In [50]:
#extract features
fracture=get_features('../dataset/train/Fracture',v4_model_trained)
normal= get_features('../dataset/train/Normal',v4_model_trained)

In [51]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [52]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [53]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.011768,0.086996,0.025058,0.014746
score_time,0.007779,0.008577,0.008580,0.014179
test_accuracy,0.808200,0.808556,0.778075,0.790909
train_accuracy,0.926247,0.813329,0.840453,1.000000
test_precision_macro,0.820714,0.837890,0.816669,0.824998
train_precision_macro,0.928986,0.813228,0.836469,1.000000
test_recall_macro,0.817556,0.818083,0.772707,0.804962
train_recall_macro,0.930904,0.820267,0.836406,1.000000
test_f1_macro,0.804903,0.803430,0.767186,0.781506
train_f1_macro,0.925132,0.812002,0.836318,1.000000


In [54]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',v4_model_trained)
test_normal= get_features('../dataset/test/Normal',v4_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [55]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [56]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.83      0.42      0.56        12
           1       0.67      0.93      0.78        15

    accuracy                           0.70        27
   macro avg       0.75      0.68      0.67        27
weighted avg       0.74      0.70      0.68        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.91      0.83      0.87        12
           1       0.88      0.93      0.90        15

    accuracy                           0.89        27
   macro avg       0.89      0.88      0.89        27
weighted avg       0.89      0.89      0.89        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.71      1.00      0.83        15

    accuracy                           0.78        